This brief example shows how to use the isochrones package to extract an isochrone for a specific location and time. 

It assumes that 
* the isochrones package has been properly installed and additional dependencies (matplotlib, numpy) have been installed (e.g. with `pip install isochrones[example]`)
* OpenTripPlanner (OTP) is running locally on the 8080 port
* the OTP data is stored in the `otp/graphs/main` subfolder

In [ ]:
from matplotlib import pyplot as plt
from matplotlib import colors as colors
import datetime
import contextily as ctx


from isochrones import calculate_isochrones, intersect_isochrones, get_osm_features

In [ ]:
# Define parameters
lat = 46.20730  # Charmilles, Geneva
lon = 6.12686
cutoffSec = [1800, 3600, 5400]  # 30, 60, 90 minutes
date_time = datetime.datetime(2022, 5, 20, 16, 0)
otp_url = f"http://localhost:8880"
router = "default"
#router = "ara_geneve_graphe_diff_vitesse"

# POI parameters
bounding_box = (
    lon - 0.3,
    lat - 0.3,
    lon + 0.3,
    lat + 0.3,
)

# bounding_box = (
#     5.882835,
#     46.068877,
#     6.337395,
#     46.325682,
# )  # from bbxofinder.com, Collonges - Fillinges, Cessy - La roche sur Foron

tags = {
    "amenity": True,
    "healthcare": True,
    "office": True,
    "public_Transport": True,
    "shop": True,
    "tourism": True,
}

# Isochrone styling
isochrone_colors = {
    1800: {"color": "#440154", "label": "30 min"},
    3600: {"color": "#21918c", "label": "60 min"},
    5400: {"color": "#fde725", "label": "90 min"},
}

# Additional POI styling
poi_categories = {
    "amenity": {"color": "red", "label": "Amenities"},
    "healthcare": {"color": "blue", "label": "Healthcare"},
    "office": {"color": "green", "label": "Offices"},
    "public_Transport": {"color": "purple", "label": "Public Transport"},
    "shop": {"color": "orange", "label": "Shops"},
    "tourism": {"color": "brown", "label": "Tourism"},
}

# Create a normalized colormap based on cutoffSec values
norm = colors.Normalize(vmin=min(cutoffSec), vmax=max(cutoffSec))
cmap = plt.cm.viridis

In [ ]:
colors.rgb2hex([0.993248, 0.906157, 0.143936])

In [ ]:
isochrone_colors = {
    1800: {"color": "#440154", "label": "30 min"},
    3600: {"color": "#21918c", "label": "60 min"},
    5400: {"color": "#fde725", "label": "90 min"},
}

In [ ]:
cmap(norm(cutoffSec))

In [ ]:
output = calculate_isochrones(
    lat, lon, cutoffSec, date_time, otp_url, router
)

In [ ]:
output.crs = "EPSG:4326"

In [ ]:
output.total_bounds

In [ ]:
def plot_with_basemap(gdf, pois=None, figsize=(15, 15), zoom=12, crs="EPSG:3857"):
    # Convert to Web Mercator for contextily
    gdf_web = gdf.to_crs(crs)

    # Create figure
    fig, ax = plt.subplots(figsize=figsize)

    # Store handles for legends
    iso_handles = []
    poi_handles = []

    # Plot isochrones with categorical colors
    for time, style in isochrone_colors.items():
        mask = gdf_web["time"] == time
        if mask.any():
            gdf_web[mask].plot(ax=ax, color=style["color"], alpha=0.5)
            iso_handles.append(
                plt.Rectangle(
                    (0, 0), 1, 1, fc=style["color"], alpha=0.5, label=style["label"]
                )
            )

    # Plot POIs with categorical colors
    if pois is not None:
        pois_web = pois.to_crs(crs)
        for category, style in poi_categories.items():
            cat_pois = pois_web[pois_web["variable"] == category]
            if not cat_pois.empty:
                cat_pois.plot(
                    ax=ax, color=style["color"], marker="o", markersize=5, alpha=0.7
                )
                poi_handles.append(
                    plt.Line2D(
                        [0],
                        [0],
                        marker="o",
                        color=style["color"],
                        markersize=8,
                        linestyle="",
                        label=style["label"],
                    )
                )

    # Add basemap
    ctx.add_basemap(
        ax,
        crs=gdf_web.crs.to_string(),
        source=ctx.providers.SwissFederalGeoportal.NationalMapColor,
        zoom=zoom,
    )

    # Set bounds and remove axes
    minx, miny, maxx, maxy = gdf_web.total_bounds
    ax.set_xlim(minx, maxx)
    ax.set_ylim(miny, maxy)
    ax.set_axis_off()

    # Add dual legends
    if iso_handles:
        first_legend = ax.legend(
            handles=iso_handles,
            title="Travel Time",
            loc="upper left",
            frameon=True,
            framealpha=0.8,
        )
        ax.add_artist(first_legend)

    if poi_handles:
        ax.legend(
            handles=poi_handles,
            title="Points of Interest",
            loc="upper right",
            frameon=True,
            framealpha=0.8,
        )

    return fig, ax

In [ ]:
# Create plot with base layer
fig, ax = plot_with_basemap(output, zoom=14)
plt.show()

In [ ]:
pois = get_osm_features(bounding_box, tags)

In [ ]:
intersected_pois = intersect_isochrones(output, pois.to_crs("EPSG:4326"))

In [ ]:
intersected_pois = intersected_pois[intersected_pois["time"] == 5400]

In [ ]:
intersected_pois

In [ ]:
fig, ax = plot_with_basemap(output, pois=intersected_pois, zoom=14)
plt.show()